# MQ Spectrograph Pipeline Plotting Lab

This notebook provides users with an insight into how to plot the raw extracted spectra, the pixel to pixel sensitivity for each order, and the subsequent smoothed spectrum. You should expect 80x3 plots per extracted data (80 raw spectra, 80 pixel to pixel plots, and 80 smoothed spectra).

-----------------------------------------------------------------------------------------------------------------

<font color = red> NOTE: the basic reduction pipeline notebook must be completed first before attempting to plot anything.
    
<font color = red> ISSUE: flux_load import does not work on PC computers - need to transfer this code all into the main reduction notebook.

## Step 1: Import Functions
Run the following code which imports a set of functions and Python packages that are utilised in the following
code. Note that the location of this file should exist in the same directory as the rest of the reduction scripts, as should your Python Notebook.

In [1]:
%run /Users/Jacob/Desktop/mq_spectrograph_final/import_functions.py # Mac example
# %run C:\\Users\\87463547\\Documents\\Physics_data\\import_functions.py # Windows example

## Step 2: Define Variables
Here we define our path to which the code looks for our extracted data to be plotted in this notebook. We also set up our plots directory by searching for a series of sub-directories, and if they do not exist, generate new sub-directories to which our plots that are generated, are saved to.

<font color = red> NOTE: remember that the naming convention used when searching for data may differ!
    
<font color = red> NOTE: flux_load needs to exist in your directory first, and is generated in the basic reduction notebook.

In [2]:
## double check what files are being imported !!!
plot_path = '/Users/Jacob/Desktop/data_for_mq_final/'
plot_list = sorted(glob.glob(plot_path + '*00_quick_extracted.fits'))

In [3]:
## set a variable to an array that you can use to iterate over in your loops
stellar_object = np.arange(0,len(plot_list))

## start the time - only for the purpose of seeing how long your plotting goes for
start_time = time.time()

## generated in basic_reduction_script - loads locally saved extracted flux dictionary 
## (basic_reduction_script.py only needs to be run once)
flux_load = np.load(plot_path + 'flux.npy') .item()

## this applies a default gaussian filter to an observed flat field in order to 
## determine the pixel-to-pixel sensitivity variations as well as the fringing pattern in the red orders. 
## this is done in 1D, ie for the already extracted spectrum.
smoothed_flat, pix_sens = onedim_pixtopix_variations(flux_load)

In [5]:
## if statement below should be false first time code is run, so directories are made 
## (/plots/ is generated in reduction notebook)
## any subsequent re-run of the code will skip the if statement

## Mac
if os.path.isdir(plot_path + '/plots/raw_extracted_spectra') == False:
    os.mkdir(plot_path + 'plots/raw_extracted_spectra')
    os.mkdir(plot_path + 'plots/pixel_sensitivity')
    os.mkdir(plot_path + 'plots/smoothed_extracted_spectra')

## this generates a sub-directory within the plots folder that are named appropriately as per the names of the
## extracted image files
for j in stellar_object:
    plot_name = plot_list[j].split('/')
    if os.path.isdir(plot_path + '/plots/raw_extracted_spectra/' + plot_name[-1].split('.')[0]) == False:
        os.mkdir(plot_path + 'plots/raw_extracted_spectra/' + plot_name[-1].split('.')[0])
    if os.path.isdir(plot_path + '/plots/pixel_sensitivity/' + plot_name[-1].split('.')[0]) == False:
        os.mkdir(plot_path + 'plots/pixel_sensitivity/' + plot_name[-1].split('.')[0])
    if os.path.isdir(plot_path + '/plots/smoothed_extracted_spectra/' + plot_name[-1].split('.')[0]) == False:
        os.mkdir(plot_path + 'plots/smoothed_extracted_spectra/' + plot_name[-1].split('.')[0])
    
## Windows
#if os.path.isdir(plot_path + 'plots\\raw_extracted_spectra') == False:
#    os.mkdir(plot_path + 'plots\\raw_extracted_spectra')
#    os.mkdir(plot_path + 'plots\\pixel_sensitivity')
#    os.mkdir(plot_path + 'plots\\smoothed_extracted_spectra')

## Step 3: Plotting
Here we plot the raw extracted spectra, the pixel to pixel sensitivity that can be used to then generate the third set of plots, the smoothed extracted spectra.

In [6]:
## iterate over all elements imported from file
for j in stellar_object:

    solar_spectrum = pyfits.getdata(plot_list[j])
    nx,ny = solar_spectrum.shape
    plot_name = plot_list[j].split('/')

    print("Plotting: " + plot_name[-1].split('/')[0])

    ## iterate over each row of each imported file
    for i, o in enumerate(sorted(flux_load.keys())):
        one_dim_solar = solar_spectrum[i, :] # cuts imported file into each 1-dim spectra
        nx = np.arange(0, ny) # set to the length of the detector (size:4096x4096)

        ############################################# RAW SPECTRUM ####################################################

        ## 1-dim raw extracted spectra with 5th order polynomial fit (other orders have worsened fit)
        ## (x - axis: pixels, y-axis: intensity)

        save_plots = plot_path + 'plots/raw_extracted_spectra/' + plot_name[-1].split('.')[0] + '/' # for plotting
        ## calculate polynomials
        z = np.polyfit(nx, one_dim_solar, 5)
        f = np.poly1d(z)
        ## calculate new x's and new y's
        x_new = np.linspace(nx[0], nx[-1], len(nx))
        y_new = f(x_new)
        ## plot
        plt.figure()
        plot_name = plot_list[j].split('/')
        plt.xlim(0, ny)
        plt.xlabel('Pixels')
        plt.ylabel('Intensity')
        plt.title(plot_name[-1].split('.')[0] + ' : ' + 'Order: ' + str(i+2))
        plt.plot(nx, one_dim_solar, x_new, y_new, 'r', linewidth=2)
        plt.savefig(save_plots + plot_name[-1].split('.')[0] + '_order_' + str(i+2) + '.png')
        #plt.show() # here for debugging
        plt.close()

        ############################################ PIXEL SENSITIVITY ################################################

        save_plots = plot_path + 'plots/pixel_sensitivity/' + plot_name[-1].split('.')[0] + '/' # for plotting
        ## find the pixel sensitivity for each order
        ## plot
        plt.figure()
        plt.xlim(0, ny)
        plt.xlabel('Pixels')
        plt.ylabel('Relative Sensitivity')
        plt.title('Pixel Sensitivity : ' + 'Order: ' + str(i+2))
        plt.plot(nx, pix_sens[o])
        plt.savefig(save_plots + plot_name[-1].split('.')[0] + '_pixel_sens_order_' + str(i+2) + '.png')
        #plt.show() # here for debugging
        plt.close()

        ############################################# SMOOTH SPECTRUM #################################################

        save_plots = plot_path + 'plots/smoothed_extracted_spectra/' + plot_name[-1].split('.')[0] + '/' # for plotting
        ## 1-dim raw extracted spectra divided by pixel sensitivity, to produce a 'smoother' spectrum
        smooth_1dim = one_dim_solar / pix_sens[o]
        ## plot
        plt.figure()
        plt.xlim(0, ny)
        plt.xlabel('Pixels')
        plt.ylabel('Intensity')
        plt.title('Smoothed: ' + plot_name[-1].split('.')[0] + ' : ' + 'Order: ' + str(i+2))
        plt.plot(nx, smooth_1dim, x_new, y_new, 'r', linewidth=2)
        plt.savefig(save_plots + plot_name[-1].split('.')[0] + '_smoothed_order_' + str(i+2) + '.png')
        #plt.show() # here for debugging
        plt.close()

        #break # can uncomment the break to test for the 2nd order of first element (need to set i=j=0 before running code)
    # break # can uncomment the break to test for the 2nd order of first element (need to set i=j=0 before running code)

print('Elapsed time: '+str(time.time() - start_time)+' seconds')

Plotting: corrected_mq_blackbody2500_quick_extracted.fits


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: divide by zero encountered in true_divide
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in true_divide


Plotting: corrected_mq_blackbody3500_quick_extracted.fits
Plotting: corrected_mq_blackbody4500_quick_extracted.fits
Plotting: corrected_mq_blackbody6500_quick_extracted.fits
Elapsed time: 219.33713221549988 seconds
